In [117]:
import pandas as pd
import string
import nltk
import torch
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
# nltk.download('stopwords')
# nltk.download('punkt') # Download the tokenizer model
# nltk.download('wordnet') # Download the wordnet corpora

data_dir = "./data/"

In [88]:
f_train = open(data_dir + "train.ende.src", "r")
train = f_train.readlines()

In [124]:
def tokenize(strings):
    tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
    return [tokenizer.tokenize(l) for l in strings]


def remove_stopwords(list_strings, additional_stopwords=(), language='english'):
    words = stopwords.words('english')
    for word in additional_stopwords:
        if word not in words: words.append(word)

    ret = []
    for sentence in list_strings:
        ret.append(list(filter(lambda word: word not in words, sentence)))
    return ret

def remove_capitalisation(list_strings):
    ret = []
    for sentence in list_strings:
        ret.append([w.lower() for w in sentence])
    return ret

def remove_punctuation(list_strings):
    ret = []
    for sentence in list_strings:
        ret.append(list(filter(lambda word: word not in string.punctuation, sentence)))
    return ret


def build_vocab(list_string):
    vocab_set = set()
    for sentence in list_string:
        vocab_set.update(sentence)
        
    return vocab_set

def build_w2i_i2w(vocabulary):
    word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
    idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}
    return word2idx, idx2word

def lemmatise():
    pass

def stem():
    pass

def extract_context(list_string, word2idx, window_size=2):
    idx_pairs = []
    for sentence in list_string:
        indices = [word2idx[word] for word in sentence]

        for center_word_pos in range(len(indices)):

            for w in range(-window_size, window_size + 1):
                context_word_pos = center_word_pos + w

                if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                    continue

                context_word_idx = indices[context_word_pos]
                idx_pairs.append((indices[center_word_pos], context_word_idx))

    return np.array(idx_pairs) # it will be useful to have this as numpy array


def get_one_hot(vocabulary, word_idx):
    x = torch.zeros(len(vocabulary)).float()
    x[word_idx] = 1.0
    return x

In [104]:
train_tokenized = tokenize(train)
train_nostop = remove_stopwords(train_tokenized)
train_nopunct = remove_punctuation(train_nostop)
train_nocap = remove_capitalisation(train_nopunct)

In [116]:
voc = build_vocab(train_nocap)
word2idx, idx2word = build_w2i_i2w(voc)
get_one_hot(voc, word2idx["how"])

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [125]:
contexts = extract_context(train_nocap, word2idx, window_size=3)

In [127]:
len(contexts)

328830